---
# 해당 코드는 천재교육 빅데이터 7기 프로젝트로 시행한 CNN 모델 구축입니다
---

- 빅데이터 이미지 처리 강의를 수강하며 진행한 프로젝트 ( 5/8 ~ 5/10)
- 1조 조장을 맡아 팀을 인솔, 자체 CNN모델을 구축
- 우리 조는 자체 CNN 모델을 구축하고, ResNet18을 이용한 두 가지 형상을 서로 비교하였다
- 캐글의 영화 포스터 이미지 분류 사용
- https://www.kaggle.com/datasets/zulkarnainsaurav/four-genre-movie-poster-images
- 같이 함께 해준 팀원들 덕분에 voted도 받고 메달도 받았다 ! 와 !
---

- 발표 피드백
- 조금 더 진중한 모습으로 할 것 !
- ppt 안에 있는 용어들은 모두 숙지 > 지역 최적점과 글로벌 최적점은 무엇인가?
- 대명사 사용 지양 (이것, 저것)

---
The annotations are in Korean.  
If you want to see it in English, use ChatGPT!  

---

In [5]:
# 필요한 라이브러리 불러오기
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# 데이터셋 경로 및 클래스 레이블 정의
root_dir = r"YourFolderName"
class_labels = ['action', 'comedy', 'horror', 'romance'] # 데이터 라벨(target)
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'] # 이미지 확장자
crop_size = 100 # 데이터 증강을 위한 이미지 크롭 사이즈 정의

# 이미지 크롭 함수
def crop_image(image, crop_size):
    height, width = image.shape[:2]
    center_x = width // 2
    center_y = height // 2
    x1 = center_x - crop_size // 2
    x2 = center_x + crop_size // 2
    y1 = center_y - crop_size // 2
    y2 = center_y + crop_size // 2
    return image[y1:y2, x1:x2]

# 이미지 플립 함수
def flip_image(image, flip_code):
    return cv2.flip(image, flip_code)

# 이미지 회전 함수
def rotate_image(image, angle):
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, matrix, (width, height))
    return rotated_image

# 각 클래스별로 이미지 변환 수행
for class_label in class_labels:
    class_dir = os.path.join(root_dir, class_label)
    os.makedirs(class_dir, exist_ok=True)

    for image_file in os.listdir(class_dir):
        if os.path.splitext(image_file)[1].lower() not in image_extensions:
            continue

        image_path = os.path.join(class_dir, image_file)
        image = cv2.imread(image_path)

        # 왼쪽으로 15도 회전한 이미지 저장
        rotated_left_image = rotate_image(image, -15)
        cv2.imwrite(os.path.join(class_dir, f"rotated_left{image_file}"), rotated_left_image)

        # 오른쪽으로 15도 회전한 이미지 저장
        rotated_right_image = rotate_image(image, 15)
        cv2.imwrite(os.path.join(class_dir, f"rotated_right{image_file}"), rotated_right_image)

        # 좌우 반전한 이미지 저장
        flipped_horizontal_image = flip_image(image, 1)
        cv2.imwrite(os.path.join(class_dir, f"flipped_horizontal{image_file}"), flipped_horizontal_image)

        # 크롭된 이미지 저장
        cropped_image = crop_image(image, crop_size)
        cv2.imwrite(os.path.join(class_dir, f"cropped{image_file}"), cropped_image)


In [6]:
# 이미지 데이터셋의 루트 디렉토리 정의
root_dir = r"YourFolderName"

# 레이블(장르 이름)의 목록 가져오기
labels = os.listdir(root_dir)

# 이미지 경로와 해당하는 레이블 저장 리스트 정의
image_paths = []
labels_list = []

# 각 레이블(장르)에 대해 반복
for label in labels:
    # 레이블 디렉토리의 경로 가져옴
    label_dir = os.path.join(root_dir, label)
    # 디렉토리가 존재하는지 확인
    if os.path.isdir(label_dir):
        # 레이블 디렉토리에 있는 이미지 파일 목록 가져오기
        images = os.listdir(label_dir)
        # 이미지 경로를 리스트에 추가
        image_paths.extend([os.path.join(label_dir, img) for img in images])
        # 해당하는 레이블을 레이블 리스트에 추가
        labels_list.extend([label] * len(images))
    else:
        print(f"Warning: Directory {label_dir} not found.")

# 데이터 확인
if len(image_paths) == 0:
    raise ValueError("No images found in the dataset.")

# 각 레이블의 비율을 유지하면서 데이터를 훈련 세트와 테스트 세트로 분할
if len(labels_list) > 0:
    train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels_list, test_size=0.2, random_state=42, stratify=labels_list)
else:
    raise ValueError("No labels found for the images.")
print("Number of training images:", len(train_paths))
print("Number of testing images:", len(test_paths))

# 각 장르의 이미지 폴더 경로 정의
train_data_dir = r"YourFolderName\train"
test_data_dir = r"YourFolderName\test"

# 훈련 및 테스트 데이터의 디렉토리 생성
os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(test_data_dir, exist_ok=True)

# 이미지를 훈련 및 테스트 디렉토리로 이동
for i in range(len(train_paths)):
    label = train_labels[i]
    image = train_paths[i]
    dst = os.path.join(train_data_dir, label, os.path.basename(image))
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    os.replace(image, dst)
for i in range(len(test_paths)):
    label = test_labels[i]
    image = test_paths[i]
    dst = os.path.join(test_data_dir, label, os.path.basename(image))
    os.makedirs(os.path.dirname(dst), exist_ok=True)
    os.replace(image, dst)

Number of training images: 5220
Number of testing images: 1305


In [ ]:
# 데이터 생성기 정의와 데이터 증강을 위한 데이터 생성기 설정
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 훈련 및 테스트 데이터셋 생성
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(224, 224),
                                                    batch_size=100,
                                                    class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(224, 224),
                                                  batch_size=1000,
                                                  class_mode='categorical')

# 모델 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(4, activation='softmax')
])

# 모델 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              metrics=['accuracy'])

# 모델 훈련
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# 클래스 레이블 가져오기
class_labels = train_generator.class_indices

# 인덱스로부터 레이블을 얻기 위해 딕셔너리 뒤집기
labels = dict((v,k) for k,v in class_labels.items())

# 이미지 예측 및 출력하는 함수 정의
def predict_images(model, generator, labels, num_images=6):
    # 데이터 생성기에서 이미지와 레이블 배치 가져오기
    image_batch, label_batch = next(generator)
    # 이미지에 대한 클래스 예측
    predictions = model.predict(image_batch)
    # 예측된 레이블과 함께 이미지 출력
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(image_batch[i])
        predicted_label = labels[np.argmax(predictions[i])]
        true_label = labels[np.argmax(label_batch[i])]
        if predicted_label == true_label:
            title_color = 'green'  # True prediction (green color)
        else:
            title_color = 'red'    # Wrong prediction (red color)
        plt.title(f"Predicted: {predicted_label}\nTrue: {true_label}", fontsize=10, color=title_color)
        plt.axis('off')
    plt.show()
    
# 함수 호출하여 예측된 이미지 출력
predict_images(model, test_generator, labels)

![-](https://github.com/PlutoJoshua/Education_CJ/blob/main/%EC%98%81%ED%99%94%ED%8F%AC%EC%8A%A4%ED%84%B0%EB%B6%84%EB%A5%98%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202024-05-13%20172355.png?raw=true)

![-](https://github.com/PlutoJoshua/Education_CJ/blob/main/%EC%98%81%ED%99%94%ED%8F%AC%EC%8A%A4%ED%84%B0%EB%B6%84%EB%A5%98%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/%EC%8A%A4%ED%81%AC%EB%A6%B0%EC%83%B7%202024-05-13%20174045.png?raw=true)

---
ResNet19은 캐글에 있습니다.  

---